#### Pre-Workflow Setup

1. Ensure you can login to the two systems we will use today: Theta@ALCF and Cori@NERSC
2. Install/Load Anaconda3 on the two target systems and create a environment with Parsl installed.
3. Create a **setup_parsl_0.7.2.sh** script with environment setup commands.
4. Goto to https://app.globus.org/endpoints and activate globus endpoints **alcf#dtn_theta** and **NERSC Cori**

Here we will load the Parsl module and print the version information

In [1]:
import parsl
from parsl.app.app import python_app, bash_app
import time
import os
from parsl import File

print("Parsl version: {}".format(parsl.__version__))
print("Parsl file source: {}".format(parsl.__file__))

Parsl version: 0.7.2-a1
Parsl file source: /home/ubuntu/anaconda3/envs/remote_ix_py3.6/lib/python3.6/site-packages/parsl/__init__.py


Enable only for logging. Please remove prior to demo time

In [ ]:
# parsl.set_stream_logger()

#### Configurations

Here we configure Parsl to use the Theta Supercomputer at Argonne National Lab, and Cori at NERSC.

##### Authentication
Both systems use two factor authentication, which Parsl supports via an interactive SSH channel that
requires the user to enter the 2FA token string as per the facilities setup when prompted.

##### Execution


##### Data Transport


In [2]:
from parsl.config import Config

from parsl.providers import SlurmProvider, CobaltProvider
from parsl.executors import HighThroughputExecutor
from parsl.channels import SSHInteractiveLoginChannel
from parsl.data_provider.scheme import GlobusScheme

config = Config(
    executors=[
        HighThroughputExecutor(
            label='theta',
            max_workers=4,            
            address='try.parsl-project.org',
            interchange_address='thetalogin6',            
            provider=CobaltProvider(                
                channel=SSHInteractiveLoginChannel(
                    hostname="thetalogin6.alcf.anl.gov",
                    username="yadunand",                         # MUST SET PER USER
                    script_dir="/home/yadunand/parsl_scripts"                    
                ),
                queue="debug-flat-quad",
                init_blocks=1,
                min_blocks=1,
                worker_init='source /home/yadunand/setup_parsl_0.7.2.sh',
                account='CSC249ADCD01',                          # MUST SET PER USER
                cmd_timeout=120
            ),
            working_dir='/home/yadunand',
            storage_access=[GlobusScheme(
                endpoint_uuid='08925f04-569f-11e7-bef8-22000b9a448b',
                endpoint_path='/',
                local_path='/')],
        ),        
        HighThroughputExecutor(
            label="cori",
            worker_debug=False,
            address='try.parsl-project.org',
            interchange_address='cori03-224.nersc.gov',            
            provider=SlurmProvider(
                partition='debug',  # Replace with partition name
                # channel=SSHChannel(
                channel=SSHInteractiveLoginChannel(
                    hostname='cori03-224.nersc.gov',
                    username='yadunand',                         # MUST SET PER USER
                    script_dir='/global/homes/y/yadunand/parsl_scripts',
                ),                
                init_blocks=1,
                min_blocks=1,
                scheduler_options="#SBATCH --constraint=knl,quad,cache",
                # scheduler_options="#SBATCH --constraint=haswell",
                worker_init='source ~/setup_parsl_0.7.2.sh',                
            ),
            working_dir='/global/homes/y/yadunand',
            storage_access=[GlobusScheme(
                endpoint_uuid='9d6d99eb-6d04-11e5-ba46-22000b92c6ec',
                endpoint_path='/',
                local_path='/')],
        )
        
    ],
)

Enter thetalogin6.alcf.anl.gov Logon password :········
Enter cori03-224.nersc.gov Logon password :········


In [3]:
parsl.load(config)

In [4]:
@python_app(executors=['theta'])
def platform_theta(sleep=10, stdout=None):
    import platform
    import time
    time.sleep(sleep)
    return platform.uname()

@python_app(executors=['cori'])
def platform_cori(sleep=10, stdout=None):
    import platform
    import time
    time.sleep(sleep)
    return platform.uname()

In [5]:
theta_p = platform_theta()
cori_p = platform_cori()

In [6]:
# print("Got result: {}".format(x.result())
print("From theta: ", theta_p.result())
print("From cori: ", cori_p.result())

From theta:  uname_result(system='Linux', node='nid00020', release='4.4.103-6.38_4.0.104-cray_ari_c', version='#1 SMP Thu Jan 17 19:44:04 UTC 2019 (5395680)', machine='x86_64', processor='x86_64')
From cori:  uname_result(system='Linux', node='nid02877', release='4.4.103-6.38_4.0.153-cray_ari_c', version='#1 SMP Thu Nov 1 16:05:05 UTC 2018 (6ef8fef)', machine='x86_64', processor='x86_64')


In [7]:
@python_app(executors=['theta'])
def write_to_file(message, outputs=[]):
    import os    
    with open(outputs[0].filepath, 'w+') as f:
        f.write(message)    
    return "Wrote to {}".format(os.path.abspath(outputs[0].filepath))


@python_app(executors=['cori'])
def append_to_file(message, inputs=[], outputs=[]):
    import os
    with open(outputs[0].filepath, 'a+') as f:
        f.write(message)   
    return "Wrote to {}".format(os.path.abspath(outputs[0].filepath))


In [ ]:
parsl.dfk().executors['theta'].working_dir = '/home/yadunand'

In [8]:
x = write_to_file("Hello from Theta\n",
                  outputs=[File("globus://9d6d99eb-6d04-11e5-ba46-22000b92c6ec//global/homes/y/yadunand/parsl_foo.out")])

In [9]:
print(x.outputs)

[<DataFuture at 0x7f48982c7dd8 state=finished returned parsl_foo.out>]


In [10]:
y = append_to_file("Hello from Cori\n",
                   inputs=[x.outputs[0]],
                   outputs=[File("globus://d9c79e7a-3abb-11e9-a613-0a54e005f950/home/ubuntu/parsl_foo.out")])

In [12]:
print(y.outputs)

[<DataFuture at 0x7f4898052588 state=finished returned parsl_foo.out>]


In [13]:
!cat /home/ubuntu/parsl_foo.out

Hello from Theta
Hello from Cori


In [ ]:
parsl.dfk().cleanup()